# Importação dos Dados

In [6]:
# Manipulação de Dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pré-processamento dos Dados
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Modelos Utilizados
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# Otimização
from sklearn.model_selection import train_test_split, GridSearchCV

# Métricas de Avaliação: Classificação
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [7]:
# Importação
dfs = pd.read_excel("../data/dados.xlsx", sheet_name = None, index_col = "Empresa")
print("DataFrames disponíveis: ", list(dfs.keys())) # representam indicadores de anos antes da falência

DataFrames disponíveis:  ['df_1y', 'df_2y', 'df_3y', 'df_4y', 'df_5y']


# Tratamento

In [8]:
def tratamento(n):
    # Selecione dados de 1 a 5 anos pré-falência: 
    df = dfs[list(dfs.keys())[n-1]]
    print("\nDataFrame Escolhido:", n, "anos pré-falência")
    
    # Apagando NA's
    df.dropna(inplace = True)
    print("\nQuantidade de Amostras:\n", df["Alvo"].value_counts())

    # Treino e Teste
    X = df.drop("Alvo", axis = 1)
    y = df["Alvo"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

    # Transformações Lineares
#     scaler = StandardScaler()
#     X_train_norm = scaler.fit_transform(X_train)
#     X_test_norm = scaler.transform(X_test)
    
#     pca = PCA()
#     X_train = pca.fit_transform(X_train_norm)
#     X_test = pca.transform(X_test_norm)
#    print("\nExplicação de Cada Componente:", pca.explained_variance_ratio_.cumsum().round(2))
    print("--------------------------------------------------")
    return X_train, X_test, y_train, y_test

# Modelagem

In [9]:
def modelagem(df, obj):
    '''
    Entradas:
    df: Tupla com X_train, X_test, y_train, y_test
    obj: String definindo o tipo de modelo (log, rf, mlp)
    
    Saídas:
    Tupla contendo f1-score, auc, acurácia, 
    '''
    
    # Desempacotando Features 
    X_train, X_test, y_train, y_test = df
    
    # Regressão Logística
    if obj == "log":
        modelo = LogisticRegression(max_iter = 1000, random_state = 42)
        modelo.fit(X_train, y_train)
        print("Modelo: Regressão Logística")
        
    # Random Forest
    if obj == "rf":
        modelo = RandomForestClassifier(random_state = 42)
        parametros = {'max_depth': [1, 5, None],
                      'min_samples_leaf': [1, 3, 5],
                      'min_samples_split': [2, 4],
                      'n_estimators': [25, 50, 100],
                      'criterion': ["gini", 'entropy']}
        
        modelo = GridSearchCV(modelo, parametros, n_jobs = -1, cv = 3, scoring = "roc_auc")
        modelo.fit(X_train, y_train)
        print("Modelo: Random Forest")
    
    # Multi-Layer Perceptron
    if obj == "mlp":
        modelo = MLPClassifier(max_iter = 1000, random_state = 42)
        parametros = {'hidden_layer_sizes': [(50,50), (50,100,50), (100,)],
                      'activation': ['tanh', 'relu'],
                      'solver': ['sgd', 'adam', "lbfgs"],
                      'alpha': [0.0001, 0.05],
                      'learning_rate': ['constant','adaptive']}

        modelo = GridSearchCV(modelo, parametros, n_jobs = -1, cv = 3, scoring = "roc_auc")
        modelo.fit(X_train, y_train)
        print("Modelo: Multi-Layer Perceptron")
        
    # Previsões
    y_pred = modelo.predict(X_test)
    
    # Métricas de Avaliação
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    print("F1-Score:", round(f1,4))
    print("AUC:", round(auc,4))
    print("Acurácia:", round(acc,4))
    print("--------------------------------------------------")
    return [f1, auc, acc, obj]

# Aplicação dos Modelos

In [10]:
tabela = []
for i in range(1,6):
    df = tratamento(i)
    for j in ["log", "rf", "mlp"]:
        linha = modelagem(df, j)
        linha.append(i)
        tabela.append(linha)


DataFrame Escolhido: 1 anos pré-falência

Quantidade de Amostras:
 0    31
1    19
Name: Alvo, dtype: int64
--------------------------------------------------
Modelo: Regressão Logística
F1-Score: 0.25
AUC: 0.475
Acurácia: 0.5385
--------------------------------------------------
Modelo: Random Forest
F1-Score: 0.7273
AUC: 0.775
Acurácia: 0.7692
--------------------------------------------------
Modelo: Multi-Layer Perceptron
F1-Score: 0.4
AUC: 0.5125
Acurácia: 0.5385
--------------------------------------------------

DataFrame Escolhido: 2 anos pré-falência

Quantidade de Amostras:
 0    32
1    19
Name: Alvo, dtype: int64
--------------------------------------------------
Modelo: Regressão Logística
F1-Score: 0.4444
AUC: 0.5952
Acurácia: 0.6154
--------------------------------------------------
Modelo: Random Forest
F1-Score: 0.6667
AUC: 0.75
Acurácia: 0.7692
--------------------------------------------------
Modelo: Multi-Layer Perceptron
F1-Score: 0.25
AUC: 0.5119
Acurácia: 0.538

In [11]:
df = pd.DataFrame(tabela)
df.rename(columns = {0: "F1-Score", 1: "AUC", 2: "Acurácia", 3: "Modelo", 4: "Anos"}, inplace = True)
df = round(df, 4)
df

,F1-Score,AUC,Acurácia,Modelo,Anos
0,0.2500,0.4750,0.5385,log,1
1,0.7273,0.7750,0.7692,rf,1
2,0.4000,0.5125,0.5385,mlp,1
3,0.4444,0.5952,0.6154,log,2
4,0.6667,0.7500,0.7692,rf,2
5,0.2500,0.5119,0.5385,mlp,2
6,0.3333,0.6000,0.6667,log,3
7,0.5714,0.7000,0.7500,rf,3
8,0.2857,0.5286,0.5833,mlp,3
9,0.5714,0.6786,0.7273,log,4


In [12]:
df.to_excel("../data/results.xlsx")